# Profitable free mobile applications for the App Store (iOS) and Google Play Markets (Android)
## 1. Background
This project is an analysis of data from free mobile applications in order to define which types of apps are more likely to attract more users.   
All the data that is used within this project is collected from two separate datasets. 
   
Data from the iOS store is collected from: ...   
Data from the Android store is collected from: ...

## 2. Data Exploration
Before analysing the entire dataset, first we explore a sample of the dataset. To do this we develop a function named `explore_data` that takes a portion of the dataset.


In [57]:
def explore_data(dataset, start = 0, end = 3, rows_and_columns = True, header = False):
    if header == True:
        print('The header is: ')
        print(dataset[start])
        print('\n')
        start += 1

    dataset_slice = dataset[start:end]

    print('The dataset is constructed as follows: ')
    
    for row in dataset_slice:
        print(row)
        print('\n') # this adds a new (empty) line after each row
    
    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0]))

Next we use the `explore_data` function to take a look at the `ios` and `android` dataset.

In [58]:
from csv import reader

ios_file = reader(open('AppleStore.csv'))
android_file = reader(open('googleplaystore.csv'))
ios = list(ios_file)
android = list(android_file)

explore_data(ios, 0, 5, rows_and_columns = True, header = True)
print('\n')
explore_data(android, 0, 5, rows_and_columns = True, header = True)

The header is: 
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


The dataset is constructed as follows: 
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows:  7198
Number of columns:  16


The header is: 
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content R

## 3. Data Cleaning
Next the data is cleaned up:
- entry 10472 needs to be removed as it's incorrect according to the [forum](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015)
- duplicate apps need to be removed
- non-English apps need to be removed
- non-free apps need to be removed

### 3.1 Remove incorrect data

In [59]:
del(android[10473]) # delete entry 10472
removed_apps = [android[10473]]

### 3.2 Remove duplicate apps

In [60]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of unique apps:' , len(unique_apps))
print('Number of duplicates', len(duplicate_apps))
print('Number of removed apps', len(removed_apps))

Number of unique apps: 9660
Number of duplicates 1181
Number of removed apps 1


To figure out why apps are listed more times than once we take one of the apps in `duplicate_apps` in android and print the duplicate results.

In [61]:
for app in android[1:]:
    name = app[0]
    if name == duplicate_apps[0]:
        print(app)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


We now see that the duplicate results are the same, except for the `Reviews` column. Likely the data was scraped at different timepoints, registering as a different entry when review numbers differ. As the entry with the most amount of reviews is likely the most recent, only this entry will be kept.

In [62]:
print('Expected length: ', len(android) - len(duplicate_apps) - len(removed_apps))

Expected length:  9659


In [83]:
# A more efficient method than the one below was developed in the next section.

# def app_selector(apps_list):
#     output = []
#     reviews = -1 # this number is set to -1 in case there would be zero reviews for the app
    
#     for app in apps_list:
#         if int(app[3]) > reviews:
#             output = app
#             reviews = int(app[3])
#     return output
        
# android_unique = []

# for item in unique_apps:
#     app_selection = []

#     for app in android[1:]:
#         name = app[0]
#         if name == item:
#             app_selection.append(app)
    
#     if app_selection != []:
#         selection = app_selector(app_selection)
#         android_unique.append(selection)

# explore_data(android_unique, 0, 3, True, False)

First we construct a dictionary named `reviews_max` to collect all the unique names as keys and the amount of reviews as variable.

In [64]:
reviews_max = {}

for app in android[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Next we used `reviews_max` to construct a new dataset `android_clean` to contain all the unique values with the maximum amount of reviews.

In [65]:
android_clean = []
already_added = []  # as some apps have duplicates with the same amount of reviews
                    # this dataset was made to account for that

for app in android[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

explore_data(android_clean, 0, 3, True, False)

The dataset is constructed as follows: 
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows:  9659
Number of columns:  13


### 3.3 Remove non-English apps
As all characters that are commonly used in English text are in the range of 0 to 127 in the [ASCII system](https://en.wikipedia.org/wiki/ASCII), a function can be determined that detects whether there are non-Enlgish characters in the app title or not. To avoid emoji's or special characters to trigger the function, a minimum of 3 characters with an ASCII code larger than 127 needs to be detected.

In [66]:
def is_english(string):
    trigger = 0
    for character in string:
        if ord(character) > 127:
            trigger += 1
    if trigger > 3:
        return False
    return True

In [67]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)

for app in ios[1:]:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
    
explore_data(android_english)
explore_data(ios_english)

The dataset is constructed as follows: 
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows:  9614
Number of columns:  13
The dataset is constructed as follows: 
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clan

### 3.4 Remove non-free apps
For this particular analysis we only take the free applications into account. Therefore any non-free apps are removed from the dataset.

In [68]:
free_android = []
free_ios = []

for app in android_english:
    price = app[7]
    if price == '0':
        free_android.append(app)

for app in ios_english:
    price = app[4]
    if price == '0.0':
        free_ios.append(app)

explore_data(free_android)
explore_data(free_ios)

The dataset is constructed as follows: 
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows:  8864
Number of columns:  13
The dataset is constructed as follows: 
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clan

## 4. Data Analysis
### 4.1 Determine most common genre
The aim of the project is to determine the kind of apps that are more likely to attract users. The next section will attempt to determine app profiles that are successful in both markets by determining the most common apps.   
The `free_android` dataset lists the type of app in column 2 named `Category` and in column 9 named `Genres`.   
The `free_ios` dataset lists the type of app in column 11 named `prime_genre`.   
To get an idea of which genre is the most popular in both stores, we'll make a frequency table using two functions. The function `freq_table()` will return a frequency table in percentages on a column. The function `display_table` will display the calculated percentages in descending order.

In [69]:
def freq_table(dataset, index):
    freq_dict = {}
    dict_percentage = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in freq_dict:
            freq_dict[value] += 1
        else:
            freq_dict[value] = 1
            
    for key in freq_dict:
        dict_percentage[key] = (freq_dict[key]/total)*100

    return dict_percentage

def display_table(dataset, index): # this function converts the dictionary in a tuple to allow sorting
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

#### **Data from iOS store**  
The code below shows the frequency table for the `prime_genre` of the `free_ios` data:

In [70]:
display_table(free_ios, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Among the free English apps, more than half (58.16%) of them are `Games`, followed by `Entertainment` apps with 7.88% and `Photo & Video` apps with 4.97%.   
The fewest apps are apps in the genre `Catalogs` with 0.12%, followed by `Medical` apps with 0.10% and `Navigation` apps with 0.19%.   
It appears as if the iOS app store is largely dominated by apps that are designed for fun (Games, Entertainment, Photo & Video), while apps for practical purposes (Catalogs, Navigation, Medical) are more rare.   
However, although the apps designed for fun are more common, does not apply that they also have the greatest number of users.


#### **Data from Google Play store**  
The code below shows the frequency table for the `Category` of the `free_android` data:

In [71]:
display_table(free_android, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The data here appears to be distributed more. Gaming-apps however maintain to be a top performer with 9.72% of all apps being in this category. The top category is `Family`, however it is important to note that a large part of this category are games for kids.
Nonetheless, practical apps appear to have a better representation on the Google Play store when comparing to the iOS App store.   
The `Genres` column of the Google Play store appears to confirm this hypothesis.

In [72]:
display_table(free_android, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Furthermore, there are far more categories on the Google Play store in the `Genres` column. For practical purposes we therefore continue with the `Category` data from the `free_android` dataset

### 4.2 Determine most popular genre
#### **Data from iOS store**
In order to determine the most popular genre the following section determines what the average number of installs if for each category (genre). As this data is not present in `free_ios`, the total number of user ratings is taken as a proxy (`rating_count_tot`).
Below a table is generated to display the average amount of user reviews per application genre in the ios_store.

In [73]:
unique_genres = {}
avg_usr = {}

for row in free_ios:
    prime_genre = row[11]
    if prime_genre in unique_genres:
        unique_genres[prime_genre] += 1
    else:
        unique_genres[prime_genre] = 1
        
for genre in unique_genres:
    total = 0
    len_genre = 0
    for app in free_ios:
        genre_app = app[11]
        if genre_app == genre:
            usr_rating = float(app[5])
            total += usr_rating
            len_genre += 1
    avg_usr[genre] = total/len_genre

table_display = []

for key in avg_usr:
    key_val_as_tuple = (avg_usr[key], key)
    table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
for entry in table_sorted:
        print(entry[1], ':', entry[0])

Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


On average Navigation apps have the highest amount of user reviews per application. This results is heavily skewed by Waze and Google Maps, which have close to half a million user reviews together.

In [74]:
for app in free_ios:
    if app[11] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same is true for social networks, with certain applications such as Facebook, Pinterest, Skype, etc. to have a large number of reviews. Nonetheless there are a great number of apps in this particular category.

In [75]:
for app in free_ios:
    if app[11] == 'Social Networking':
        print(app[1], ':', app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Interestingly the genre `Reference` has a large number of average user ratings as well on average, while there are relatively few applications in this particular category highlighting a potential niche that could prove to be successful.

In [76]:
for app in free_ios:
    if app[11] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Similarly, other examples with potential are Weather, Food and Drink and Finance apps

#### **Data from Google Play store**
The install numbers of the Google Play store are often open-ended. Therefore the numbers will likely be incorrect as it is not known whether an app with 10,000+ installs has 11,000, 20,000 or 40,000 installs. As perfect precision is not needed for the purpose of this project, this is left unchanged. Therefore an app with 10,000+ installs will have only 10,000 installs in the used dataset.

In [77]:
display_table(free_android, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


In [78]:
category_column = {}
avg_usr = {}

for app in free_android:
    category = app[1]
    if category in category_column:
        category_column[category] += 1
    else:
        category_column[category] = 1

for category in category_column:
    total = 0
    len_category = 0
    for app in free_android:
        if app[1] == category:
            installs = app[5].replace('+','')
            installs = installs.replace(',', '')
            installs = float(installs)
            total += installs
            len_category += 1
    avg_usr[category] = total/len_category
    
table_display = []

for key in avg_usr:
    key_val_as_tuple = (avg_usr[key], key)
    table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
for entry in table_sorted:
        print(entry[1], ':', round(entry[0]))

COMMUNICATION : 38456119
VIDEO_PLAYERS : 24727872
SOCIAL : 23253652
PHOTOGRAPHY : 17840110
PRODUCTIVITY : 16787331
GAME : 15588016
TRAVEL_AND_LOCAL : 13984078
ENTERTAINMENT : 11640706
TOOLS : 10801391
NEWS_AND_MAGAZINES : 9549178
BOOKS_AND_REFERENCE : 8767812
SHOPPING : 7036877
PERSONALIZATION : 5201483
WEATHER : 5074486
HEALTH_AND_FITNESS : 4188822
MAPS_AND_NAVIGATION : 4056942
FAMILY : 3695642
SPORTS : 3638640
ART_AND_DESIGN : 1986335
FOOD_AND_DRINK : 1924898
EDUCATION : 1833495
BUSINESS : 1712290
LIFESTYLE : 1437816
FINANCE : 1387692
HOUSE_AND_HOME : 1331541
DATING : 854029
COMICS : 817657
AUTO_AND_VEHICLES : 647318
LIBRARIES_AND_DEMO : 638504
PARENTING : 542604
BEAUTY : 513152
EVENTS : 253542
MEDICAL : 120551


It can be determined that the `COMMUNICATION` category has the most number of installs, with an average of 38,456,119. This result is significantly skewed by WhatsApp Messenger with over 1,000,000,000 installs. Nonetheless, there are a great number of other applications in the same category.

In [79]:
for app in free_android:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+' or
                                           app[5] == '500,000,000+' or 
                                           app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

A similar pattern is visible in the `VIDEO_PLAYERS` category, with the app YouTube significantly skewing the result with over 1,000,000,000 installs. Again, there are a great number of other applications in this particular category.

In [80]:
for app in free_android:
    if app[1] == 'VIDEO_PLAYERS' and (app[5] == '1,000,000,000+' or
                                           app[5] == '500,000,000+' or 
                                           app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

YouTube : 1,000,000,000+
Motorola Gallery : 100,000,000+
VLC for Android : 100,000,000+
Google Play Movies & TV : 1,000,000,000+
MX Player : 500,000,000+
Dubsmash : 100,000,000+
VivaVideo - Video Editor & Photo Movie : 100,000,000+
VideoShow-Video Editor, Video Maker, Beauty Camera : 100,000,000+
Motorola FM Radio : 100,000,000+


When taking a category in the middle of the pack a greater distribution of the install numbers appears to manifest with only a couple of apps to be very popular.

In [81]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+' or
                                           app[5] == '500,000,000+' or 
                                           app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


Adjusting the criteria in this category shows that there are different niches within this category that still have quite some potential. In particular ebook readers and ebook versions of popular genres appear to be very popular.

In [82]:
for app in free_android:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+' or
                                           app[5] == '5,000,000+' or 
                                           app[5] == '10,000,000+' or 
                                           app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

## 5. Conclusions
In this project, data about the iOS app store and the Google Play store was analysed with the goal of determining a free English app profile that has a healthy profitability in both markets.

In both markets, it appears as if an application that revolves around a popular book or book series would have some potential. Although there are a great variety of dictionaries and ebook readers, specific genres appear to be rather underpresented although still high in demand.